In [1]:
import csv
import time
from pywifi import PyWiFi, const

def scan_wifi():
    wifi = PyWiFi()
    iface = wifi.interfaces()[0]  # Get the first wireless interface

    iface.scan()  # Start scanning
    time.sleep(3)  # Wait for scanning to complete

    results = iface.scan_results()
    networks = []

    for network in results:
        network_info = {
            "SSID": network.ssid,
            "BSSID": network.bssid,
            "Signal": network.signal,
            "Frequency": network.freq,
            "Auth": network.auth,
            "AKM": network.akm
        }
        networks.append(network_info)

    return networks

def save_to_csv(networks, filename="wifi_scan_results.csv"):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ["X Coordinate", "Y Coordinate", "Z Coordinate", "SSID", "BSSID", "Signal", "Frequency", "Auth", "AKM"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for network in networks:
            writer.writerow(network)

In [4]:
if __name__ == "__main__":
    all_networks = []

    while True:
        x = input("Enter X coordinate (type 'quit' and press Enter to exit): ")
        if x.lower() == 'quit':
            break

        y = input("Enter Y coordinate: ")
        z = input("Enter Z coordinate: ")

        networks = scan_wifi()
        for network in networks:
            network.update({"X Coordinate": x, "Y Coordinate": y, "Z Coordinate": z})
        all_networks.extend(networks)
    
    save_to_csv(all_networks)
    print("WiFi scan results saved to 'wifi_scan_results.csv'.")

Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 0
Enter Z coordinate: 0
Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 0
Enter Z coordinate: 5
Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 50
Enter Z coordinate: 0
Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 50
Enter Z coordinate: 5
Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 100
Enter Z coordinate: 0
Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 100
Enter Z coordinate: 5
Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 150
Enter Z coordinate: 0
Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 150
Enter Z coordinate: 5
Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 200
Enter Z coordinate: 0
Enter X coordinate (type 'quit' and press Enter to e

pywifi 2023-12-11 16:47:27,950 ERROR Open handle failed!


Enter X coordinate (type 'quit' and press Enter to exit): 0
Enter Y coordinate: 260
Enter Z coordinate: 0


pywifi 2023-12-11 16:47:44,116 ERROR Open handle failed!


Enter X coordinate (type 'quit' and press Enter to exit): quit
WiFi scan results saved to 'wifi_scan_results.csv'.


In [5]:
import csv
from scipy.optimize import least_squares

def read_wifi_data(filename="wifi_scan_results.csv"):
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        wifi_data = list(reader)
    return wifi_data

wifi_data = read_wifi_data()

def organize_data_by_bssid(wifi_data):
    bssid_data = {}
    for entry in wifi_data:
        bssid = entry['BSSID']
        if bssid not in bssid_data:
            bssid_data[bssid] = {'coordinates': [], 'signals': []}
        coords = [float(entry['X Coordinate']), float(entry['Y Coordinate']), float(entry['Z Coordinate'])]
        signal = abs(int(entry['Signal']))  # Convert signal to a positive value
        bssid_data[bssid]['coordinates'].append(coords)
        bssid_data[bssid]['signals'].append(signal)
    return bssid_data

bssid_data = organize_data_by_bssid(wifi_data)

def trilateration(nodes, distances):
    def equations(point):
        x, y, z = point
        return [((x - nx)**2 + (y - ny)**2 + (z - nz)**2 - d**2) for [nx, ny, nz], d in zip(nodes, distances)]
    initial_guess = [0, 0, 0]
    result = least_squares(equations, initial_guess)
    return result.x

estimated_positions = {}
for bssid, data in bssid_data.items():
    if len(data['coordinates']) >= 3:  # Ensure there are at least 3 points for trilateration
        position = trilateration(data['coordinates'], data['signals'])
        estimated_positions[bssid] = position

# Save the estimated positions to a new CSV file
with open('estimated_positions.csv', 'w', newline='') as csvfile:
    fieldnames = ['BSSID', 'X Position', 'Y Position', 'Z Position']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for bssid, position in estimated_positions.items():
        writer.writerow({'BSSID': bssid, 'X Position': position[0], 'Y Position': position[1], 'Z Position': position[2]})